In [15]:
import pandas as pd 
import numpy as np

df = pd.read_excel(r"C:\Users\wallace.souza\Downloads\LOGISTICA 2.0 - CONTROLE FINANCEIRO DE LOGÍSTICA.xlsx")
fn = pd.read_excel(r"C:\Users\wallace.souza\Downloads\Fornecedores.xlsx")


In [16]:
df = df.merge(fn, on='Código Fornecedor:', how='left')

In [17]:
df = df[df['Hora de início'].notnull()]

In [18]:
pd.set_option('display.max_columns', None)


In [19]:
df = df[['Hora de conclusão','Setor:\n',
       'Total de Pallets:','Total de Volumes:',
       'Filmagem de Quantos Palets ?','Total de Palets MISTURADOS?','Lona Truck ?','Lona Carreta ?',
       'Total $ Recebido em CASH?', 'Total Recebido por PIX ?',
       'Valor total dos Serviços ?', 'Valor Pago:', 'Valor da Diferença:',
      'Código Fornecedor:','Nome do Fornecedor:_x','Nome do Fornecedor:_y', 'Nome da Transportadora:',    
       'Data do Recebimento dos Valores:', 'Data do Serviço:',
       'Constava agendamento para hoje:',
        'Reg']]

df = df.rename(columns={'Nome do Fornecedor:_x':'Nome do Fornecedor: - Ivo'})
df = df.rename(columns={'Nome do Fornecedor:_y':'Nome do Fornecedor:'})

In [20]:
df['Valor Pago:'] = pd.to_numeric(df['Valor Pago:'], errors='coerce')


In [21]:
df['Valor total dos Serviços ?'] = pd.to_numeric(df['Valor total dos Serviços ?'], errors='coerce')


In [22]:
df[df['Valor Pago:'].isna()]
df['Nome do Fornecedor:'] = df['Nome do Fornecedor:'].str.strip()

In [23]:
df['%'] = df['Valor Pago:'] / df['Valor total dos Serviços ?'] * 100 - 100
df['Taxa'] = np.where(df['Total Recebido por PIX ?'] != 0, 'Taxa Pix?', "Ok")
df['Total de Pallets:'] = df['Total de Pallets:'].astype(float)
df['Total de Volumes:'] = df['Total de Volumes:'].astype(float)
df['Filmagem de Quantos Palets ?'] = df['Filmagem de Quantos Palets ?'].astype(float)

peso_pallet = np.where(df['Setor:\n'] == 'Perecíveis', 30, 25)
fornecedor = np.where(df['Nome do Fornecedor:'] == 'AMBEV S/A',20, peso_pallet)
lona = np.where(df['Lona Carreta ?'] == 'Sim',70, 0)
deslonamento = np.where(df['Lona Truck ?'] == 'Sim',50, 0)

df['Total de Volumes:']
	

denominador = (
        df['Total de Pallets:'] * fornecedor +
        df['Total de Volumes:'] + deslonamento +
        df['Filmagem de Quantos Palets ?'] * 20 +
        df['Total de Palets MISTURADOS?'] * 60 + lona
        
    )
    
# Cálculo da coluna final
df['Valor Pallet'] = df['Valor total dos Serviços ?'] / denominador
df['Valor Correto'] = denominador


In [24]:
colunas = ['Total $ Recebido em CASH?', 'Total Recebido por PIX ?',
           'Valor total dos Serviços ?', 'Valor Pago:', 'Valor da Diferença:']

df[colunas] = df[colunas].apply(
    lambda col: pd.to_numeric(col, errors='coerce')
)


In [25]:
df['Diferença'] = df['Valor total dos Serviços ?'] - df['Valor Correto']
df['Pix + Cash'] = (df['Total $ Recebido em CASH?'] + df['Total Recebido por PIX ?']) / df['Valor Pago:']


In [26]:
df['Data'] = df['Data do Recebimento dos Valores:'].dt.month

In [27]:
df1 = df[df['Data']== 11]

In [28]:
df1['Reg'] = df1['Reg'].astype(int) 
df1['Código Fornecedor:'] = df1['Código Fornecedor:'].astype(int) 

C:\Users\wallace.souza\AppData\Local\Temp\ipykernel_40468\2778285773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Reg'] = df1['Reg'].astype(int)
C:\Users\wallace.souza\AppData\Local\Temp\ipykernel_40468\2778285773.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Código Fornecedor:'] = df1['Código Fornecedor:'].astype(int)


In [29]:
df1[(df1['Valor Pallet'] != 1) | (df1['Pix + Cash'] != 1) | (df1["Nome do Fornecedor:"].isna())]

,Hora de conclusão,Setor:\n,Total de Pallets:,Total de Volumes:,Filmagem de Quantos Palets ?,Total de Palets MISTURADOS?,Lona Truck ?,Lona Carreta ?,Total $ Recebido em CASH?,Total Recebido por PIX ?,Valor total dos Serviços ?,Valor Pago:,Valor da Diferença:,Código Fornecedor:,Nome do Fornecedor: - Ivo,Nome do Fornecedor:,Nome da Transportadora:,Data do Recebimento dos Valores:,Data do Serviço:,Constava agendamento para hoje:,Reg,%,Taxa,Valor Pallet,Valor Correto,Diferença,Pix + Cash,Data
18714,2025-11-06 17:04:13,Mercearia,60.0,0.0,0.0,0.0,Não,Não,1800.0,0.0,1500.0,1800.0,0.0,1845,MFL,NaN,PROPRIO,2025-11-06,2025-11-06,Não,18716,20.0,Ok,1.000000,1500.0,0.0,1.0,11
19024,2025-11-21 16:10:37,Perecíveis,24.0,0.0,0.0,0.0,Não,Não,0.0,660.0,600.0,660.0,0.0,2457703,FRIGO RIO,"FRIGO RIO COMERCIO DE ALIMENTOS, MATERIAIS E T...",PROPRIO,2025-11-21,2025-11-21,Sim,19026,10.0,Taxa Pix?,0.833333,720.0,-120.0,1.0,11
19027,2025-11-21 16:47:35,Perecíveis,18.0,0.0,0.0,0.0,Não,Não,0.0,594.0,540.0,540.0,0.0,1006,BRF,BRF S.A.,PROPRIO,2025-11-21,2025-11-21,Sim,19029,0.0,Taxa Pix?,1.000000,540.0,0.0,1.1,11
19028,2025-11-21 16:48:36,Perecíveis,4.0,0.0,0.0,3.0,Não,Não,0.0,264.0,240.0,264.0,0.0,2637465,FORNO DE MINAS,FORNO DE MINAS ALIMENTOS LTDA,PROPRIO,2025-11-21,2025-11-21,Sim,19030,10.0,Taxa Pix?,0.800000,300.0,-60.0,1.0,11
19029,2025-11-21 16:49:32,Mercearia,3.0,0.0,0.0,2.0,Não,Não,0.0,231.0,210.0,231.0,0.0,1278,FIAMMA,FIAMMA INDUSTRIA E COMERCIO DE ALIMENTOS EIRELI,PROPRIO,2025-11-21,2025-11-21,Sim,19031,10.0,Taxa Pix?,1.076923,195.0,15.0,1.0,11
19031,2025-11-24 10:22:02,Mercearia,19.0,0.0,0.0,2.0,Não,Não,0.0,690.0,690.0,690.0,0.0,1003,BRF,AMBEV S/A,PROPRIO,2025-11-24,2025-11-24,Sim,19033,0.0,Taxa Pix?,1.380000,500.0,190.0,1.0,11
19040,2025-11-24 13:05:40,Perecíveis,6.0,0.0,0.0,0.0,Não,Não,0.0,198.0,180.0,198.0,0.0,0,SATURNO,NaN,PRPPRIO,2025-11-24,2025-11-24,Sim,19042,10.0,Taxa Pix?,1.000000,180.0,0.0,1.0,11
19046,2025-11-24 16:33:40,Perecíveis,11.0,0.0,0.0,20.0,Não,Não,0.0,450.0,450.0,450.0,0.0,1006,BRF,BRF S.A.,PROPRIO,2025-11-24,2025-11-20,Sim,19048,0.0,Taxa Pix?,0.294118,1530.0,-1080.0,1.0,11
19047,2025-11-24 17:04:41,Mercearia,2.0,0.0,0.0,0.0,Não,Não,0.0,520.0,520.0,520.0,0.0,1003,AMBEV,AMBEV S/A,FADEL,2025-11-24,2025-11-24,Sim,19049,0.0,Taxa Pix?,13.000000,40.0,480.0,1.0,11


In [30]:
noshow = df[(df['Constava agendamento para hoje:']=='Sim, No Show pendente') | (df['%']>=30)]
nao = df[(df['Constava agendamento para hoje:'].str.contains('não', na=False, case=False))]

In [31]:
print('pagamento em pix:' ,len(df[df['Total Recebido por PIX ?'] !=0]))
print('pagamento em dinheiro:' , len(df[df['Total Recebido por PIX ?'] ==0]))

pagamento em pix: 1548
pagamento em dinheiro: 1434


In [32]:
print('noshow = pagamento em pix:', len(noshow[noshow['Total Recebido por PIX ?'] !=0]))
print('noshow = pagamento em dinheiro:' , len(noshow[noshow['Total Recebido por PIX ?'] ==0]))

noshow = pagamento em pix: 241
noshow = pagamento em dinheiro: 50


In [33]:
print('não agendado = pagamento em pix:', len(nao[nao['Total Recebido por PIX ?'] !=0]))
print('não agendado = pagamento em dinheiro:' , len(nao[nao['Total Recebido por PIX ?'] ==0]))

não agendado = pagamento em pix: 126
não agendado = pagamento em dinheiro: 56


In [34]:
df1[df1['Data do Serviço:'] != df1['Data do Recebimento dos Valores:']]

,Hora de conclusão,Setor:\n,Total de Pallets:,Total de Volumes:,Filmagem de Quantos Palets ?,Total de Palets MISTURADOS?,Lona Truck ?,Lona Carreta ?,Total $ Recebido em CASH?,Total Recebido por PIX ?,Valor total dos Serviços ?,Valor Pago:,Valor da Diferença:,Código Fornecedor:,Nome do Fornecedor: - Ivo,Nome do Fornecedor:,Nome da Transportadora:,Data do Recebimento dos Valores:,Data do Serviço:,Constava agendamento para hoje:,Reg,%,Taxa,Valor Pallet,Valor Correto,Diferença,Pix + Cash,Data
18622,2025-11-03 11:13:51,Mercearia,41.0,0.0,0.0,0.0,Não,Não,0.0,1127.5,1025.0,1127.5,0.0,1323,ROMANATO,ROMANATO ALIMENTOS LTDA,PROPRIO,2025-11-03,2025-10-30,Sim,18624,10.0,Taxa Pix?,1.000000,1025.0,0.0,1.0,11
18675,2025-11-05 12:14:30,Mercearia,14.0,0.0,0.0,0.0,Não,Não,0.0,423.5,350.0,423.5,0.0,1093,UNILIDER,UNILIDER DISTRIBUIDORA S.A.,PROPRIO,2025-11-05,2025-11-04,Sim,18677,21.0,Taxa Pix?,1.000000,350.0,0.0,1.0,11
18837,2025-11-13 09:35:43,Perecíveis,32.0,0.0,4.0,0.0,Não,Não,1144.0,0.0,1040.0,1144.0,0.0,2676848,NEW BRÁZ,NEW BRAZ IMPORT & EXPORT LTDA,PROPRIO,2025-11-13,2025-11-04,Sim,18839,10.0,Ok,1.000000,1040.0,0.0,1.0,11
19046,2025-11-24 16:33:40,Perecíveis,11.0,0.0,0.0,20.0,Não,Não,0.0,450.0,450.0,450.0,0.0,1006,BRF,BRF S.A.,PROPRIO,2025-11-24,2025-11-20,Sim,19048,0.0,Taxa Pix?,0.294118,1530.0,-1080.0,1.0,11


In [35]:
b = df1[(df1['Taxa'] == 'Taxa Pix?') & (df1['%'] == 0)]

In [36]:
b["Nome do Fornecedor:"].value_counts().reset_index()

,Nome do Fornecedor:,count
0,AMBEV S/A,19
1,BRF S.A.,14
2,COPAPA CIA DE PAPEIS,1


In [38]:
df1[(df1["Data do Recebimento dos Valores:"].dt.day == 24)]

,Hora de conclusão,Setor:\n,Total de Pallets:,Total de Volumes:,Filmagem de Quantos Palets ?,Total de Palets MISTURADOS?,Lona Truck ?,Lona Carreta ?,Total $ Recebido em CASH?,Total Recebido por PIX ?,Valor total dos Serviços ?,Valor Pago:,Valor da Diferença:,Código Fornecedor:,Nome do Fornecedor: - Ivo,Nome do Fornecedor:,Nome da Transportadora:,Data do Recebimento dos Valores:,Data do Serviço:,Constava agendamento para hoje:,Reg,%,Taxa,Valor Pallet,Valor Correto,Diferença,Pix + Cash,Data
19031,2025-11-24 10:22:02,Mercearia,19.0,0.0,0.0,2.0,Não,Não,0.0,690.0,690.0,690.0,0.0,1003,BRF,AMBEV S/A,PROPRIO,2025-11-24,2025-11-24,Sim,19033,0.0,Taxa Pix?,1.380000,500.0,190.0,1.0,11
19032,2025-11-24 10:23:58,Mercearia,26.0,0.0,0.0,0.0,Não,Não,650.0,0.0,650.0,650.0,0.0,1533,GRANDE RIO,GR HIGIENE LIMPEZA LTDA,PROPRIO,2025-11-24,2025-11-24,Sim,19034,0.0,Ok,1.000000,650.0,0.0,1.0,11
19033,2025-11-24 10:57:42,Mercearia,3.0,0.0,0.0,0.0,Não,Não,97.5,0.0,75.0,97.5,0.0,2228341,AMERICA BEV,AMERICAN BEV DISTRIBUICAO LTDA,PROPRIO,2025-11-24,2025-11-24,"Sim, No Show pendente",19035,30.0,Ok,1.000000,75.0,0.0,1.0,11
19034,2025-11-24 10:58:45,Mercearia,4.0,0.0,0.0,0.0,Não,Não,80.0,0.0,80.0,80.0,0.0,1003,AMBEV,AMBEV S/A,COLOG,2025-11-24,2025-11-24,Sim,19036,0.0,Ok,1.000000,80.0,0.0,1.0,11
19035,2025-11-24 11:15:49,Mercearia,24.0,60.0,2.0,0.0,Não,Não,700.0,0.0,700.0,700.0,0.0,1012,COPAPA,COPAPA CIA DE PAPEIS,JS,2025-11-24,2025-11-24,Sim,19037,0.0,Ok,1.000000,700.0,0.0,1.0,11
19036,2025-11-24 11:16:38,Mercearia,26.0,0.0,0.0,0.0,Não,Não,650.0,0.0,650.0,650.0,0.0,1012,COPAPA,COPAPA CIA DE PAPEIS,JS,2025-11-24,2025-11-24,Sim,19038,0.0,Ok,1.000000,650.0,0.0,1.0,11
19037,2025-11-24 11:23:14,Mercearia,2.0,0.0,0.0,0.0,Não,Não,50.0,0.0,50.0,50.0,0.0,11529,USINA COLOMBO,USINA SAO FRANCISCO S/A,PROPRIO,2025-11-24,2025-11-24,Sim,19039,0.0,Ok,1.000000,50.0,0.0,1.0,11
19038,2025-11-24 12:13:30,Mercearia,9.0,0.0,0.0,0.0,Não,Não,0.0,247.5,225.0,247.5,0.0,1367,MAXX,MAMAO DISTRIBUIDORA DE ALIMENTOS LTDA,PROPRIO,2025-11-24,2025-11-24,Sim,19040,10.0,Taxa Pix?,1.000000,225.0,0.0,1.0,11
19039,2025-11-24 12:28:06,Perecíveis,6.0,0.0,0.0,0.0,Não,Não,180.0,0.0,180.0,180.0,0.0,1004,VIGOR,VIGOR ALIMENTOS S A,PROPRIO,2025-11-24,2025-11-24,Sim,19041,0.0,Ok,1.000000,180.0,0.0,1.0,11
19040,2025-11-24 13:05:40,Perecíveis,6.0,0.0,0.0,0.0,Não,Não,0.0,198.0,180.0,198.0,0.0,0,SATURNO,NaN,PRPPRIO,2025-11-24,2025-11-24,Sim,19042,10.0,Taxa Pix?,1.000000,180.0,0.0,1.0,11
